In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')
import os
os.getcwd()

Mounted at /content/drive


'/content/drive/MyDrive/Colab Notebooks'

In [3]:
# !unzip /content/drive/MyDrive/dlp.zip

In [4]:
# TRAIN_DIR = '/content/drive/MyDrive/Colab Notebooks/train'
# TEST_DIR = '/content/drive/MyDrive/Colab Notebooks/test'

In [5]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Conv2D, MaxPooling2D, BatchNormalization, Activation, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Define the paths to the train and test data directories
train_dir = '/content/drive/MyDrive/Colab Notebooks/train/'
test_dir = '/content/drive/MyDrive/Colab Notebooks/test/'

# Define a mapping between emotion labels and numeric labels
emotion_mapping = {
    'angry': 0,
    'disgust': 1,
    'fear': 2,
    'happy': 3,
    'sad': 4,
    'surprise': 5,
    'neutral': 6
}

# Initialize lists to store image data and labels
image_data = []
labels = []

In [6]:
# Function to load and preprocess images from a directory
def load_and_preprocess_images(directory, emotion_label_numeric):
    for image_file in os.listdir(directory):
        image_path = os.path.join(directory, image_file)

        # Load and preprocess the image
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
        image = cv2.resize(image, (48, 48))  # Resize to 48x48 pixels

        # Append the image data and label to the lists
        image_data.append(image)
        labels.append(emotion_label_numeric)

# Iterate through each folder (emotion label) in the train directory
for emotion_label in os.listdir(train_dir):
    emotion_dir = os.path.join(train_dir, emotion_label)

    # Check if it's a directory and if it exists in the emotion mapping
    if os.path.isdir(emotion_dir) and emotion_label in emotion_mapping:
        emotion_label_numeric = emotion_mapping[emotion_label]

        # Load and preprocess images from the current emotion folder
        load_and_preprocess_images(emotion_dir, emotion_label_numeric)

# Convert the lists to NumPy arrays
x_data = np.array(image_data)
y_data = np.array(labels)

# Convert labels to one-hot encoding
y_data = to_categorical(y_data, num_classes=7)

# Split the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

# Reshape the images to have 3 channels (grayscale to RGB)
x_train = np.stack((x_train,) * 3, axis=-1)
x_test = np.stack((x_test,) * 3, axis=-1)

In [7]:
# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
)

# Define the ResNet50 model from scratch
def resnet_block(x, filters, kernel_size=3, stride=1, conv_shortcut=True):
    shortcut = x
    if conv_shortcut:
        shortcut = Conv2D(filters, (1, 1), strides=stride)(shortcut)
        shortcut = BatchNormalization()(shortcut)

    x = Conv2D(filters, kernel_size, strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)

    x = tf.keras.layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x

input_shape = (48, 48, 3)
inputs = Input(shape=input_shape)

x = Conv2D(64, (7, 7), strides=2, padding='same')(inputs)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(x)

x = resnet_block(x, 64, conv_shortcut=False)
x = resnet_block(x, 64)
x = resnet_block(x, 64)

x = resnet_block(x, 128, stride=2)
x = resnet_block(x, 128)
x = resnet_block(x, 128)
x = resnet_block(x, 128)

x = resnet_block(x, 256, stride=2)
x = resnet_block(x, 256)
x = resnet_block(x, 256)
x = resnet_block(x, 256)
x = resnet_block(x, 256)

x = resnet_block(x, 512, stride=2)
x = resnet_block(x, 512)
x = resnet_block(x, 512)

x = GlobalAveragePooling2D()(x)
x = Dense(7, activation='softmax')(x)

model = Model(inputs=inputs, outputs=x)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Learning Rate Schedule
def learning_rate_schedule(epoch):
    if epoch < 10:
        return 0.001
    elif epoch < 20:
        return 0.0001
    else:
        return 0.00001

lr_scheduler = LearningRateScheduler(learning_rate_schedule)

# Train the model with early stopping and learning rate schedule
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

history = model.fit(
   datagen.flow(x_train, y_train, batch_size=128),
   epochs=50,
   validation_data=(x_test, y_test),
   callbacks=[early_stopping, lr_scheduler],
   verbose=1
)

Epoch 1/50
180/180 [==============================] - 70s 180ms/step - loss: 1.7880 - accuracy: 0.2729 - val_loss: 1.7646 - val_accuracy: 0.3194 - lr: 0.0010
Epoch 2/50
180/180 [==============================] - 30s 166ms/step - loss: 1.6551 - accuracy: 0.3428 - val_loss: 1.7900 - val_accuracy: 0.3553 - lr: 0.0010
Epoch 3/50
180/180 [==============================] - 30s 165ms/step - loss: 1.5650 - accuracy: 0.3848 - val_loss: 2.0703 - val_accuracy: 0.3474 - lr: 0.0010
Epoch 4/50
180/180 [==============================] - 31s 169ms/step - loss: 1.5002 - accuracy: 0.4131 - val_loss: 2.3232 - val_accuracy: 0.3581 - lr: 0.0010
Epoch 5/50
180/180 [==============================] - 30s 169ms/step - loss: 1.4292 - accuracy: 0.4499 - val_loss: 1.8931 - val_accuracy: 0.3993 - lr: 0.0010
Epoch 6/50
180/180 [==============================] - 29s 160ms/step - loss: 1.3758 - accuracy: 0.4723 - val_loss: 1.8131 - val_accuracy: 0.4582 - lr: 0.0010
Epoch 7/50
180/180 [==============================] 

In [10]:
# Evaluate the model and print test accuracy
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {(test_accuracy) * 100:.2f}%")

# Calculate the training accuracy
train_loss, train_accuracy = model.evaluate(x_train, y_train)
print(f"Training Loss: {train_loss:.4f}")
print(f"Training Accuracy: {(train_accuracy) * 100:.2f}%")

180/180 [==============================] - 3s 14ms/step - loss: 1.0368 - accuracy: 0.6230
Test Loss: 1.0368
Test Accuracy: 62.30%
718/718 [==============================] - 10s 14ms/step - loss: 0.9132 - accuracy: 0.6573
Training Loss: 0.9132
Training Accuracy: 65.73%


In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation loss
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(train_loss, label='Training Loss')
plt.plot(test_loss, label='Validation Loss')
plt.title('Loss Curves')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(train_accuracy, label='Training Accuracy')
plt.plot(test_accuracy, label='Validation Accuracy')
plt.title('Accuracy Curves')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()